In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nhl-game-data-20132021/teamInfo.csv
/kaggle/input/nhl-game-data-20132021/playsStats.csv
/kaggle/input/nhl-game-data-20132021/gameStats.csv
/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv
/kaggle/input/nhl-game-data-20132021/gamePlayerStats.csv
/kaggle/input/nhl-game-data-20132021/penalties.csv
/kaggle/input/nhl-game-data-20132021/gameGoalieStats.csv
/kaggle/input/nhl-game-data-20132021/gameInfo.csv
/kaggle/input/nhl-game-data-20132021/goals.csv
/kaggle/input/nhl-game-data-20132021/playPlayersStats.csv
/kaggle/input/nhl-game-data-20132021/playerInfo.csv
/kaggle/input/nhl-game-data-20132021/shifts.csv
/kaggle/input/nhl-game-data-20132021/gameOfficialsInfo.csv


<h1> Features </h1>

Team, location, FC, GF, GA, GD, PP, PK, Sh, Sv, PDO, WS, 5-5

In [3]:
basicGameStatsSeasonCumulativeDf = pd.read_csv('/kaggle/input/nhl-game-data-20132021/basicGameStatsSeasonCumulative.csv')

# One-hot encodings
teams = basicGameStatsSeasonCumulativeDf['Team']
oneHotTeams = pd.get_dummies(teams)
#print(oneHot)

location = basicGameStatsSeasonCumulativeDf['Location']
oneHotLocation = pd.get_dummies(location)
#print(oneHotLocation)

basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.drop(['Team', 'Location'], axis=1)
#basicGameStatsSeasonCumulativeDf.insert(2, 'Away', oneHotLocation['Away'])
#basicGameStatsSeasonCumulativeDf.insert(3, 'Home', oneHotLocation['Home'])
basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.join(oneHotTeams)
#print(basicGameStatsSeasonCumulativeDf)
basicGameStatsSeasonCumulativeDf = basicGameStatsSeasonCumulativeDf.join(oneHotLocation)
#print(basicGameStatsSeasonCumulativeDf)


# Change label to ints (1 = Win, 0 = Loss)
basicGameStatsSeasonCumulativeDf.loc[basicGameStatsSeasonCumulativeDf['Label'] == 'Win', 'Label'] = 1
basicGameStatsSeasonCumulativeDf.loc[basicGameStatsSeasonCumulativeDf['Label'] == 'Loss', 'Label'] = 0
print(basicGameStatsSeasonCumulativeDf)

           GameID  Fenwick Close %   GF   GA   GD        PP%        PK%  \
0      2013020001        50.000000    0    0    0  50.000000  50.000000   
1      2013020001        50.000000    0    0    0  50.000000  50.000000   
2      2013020002        50.000000    0    0    0  50.000000  50.000000   
3      2013020002        50.000000    0    0    0  50.000000  50.000000   
4      2013020003        50.000000    0    0    0  50.000000  50.000000   
...           ...              ...  ...  ...  ...        ...        ...   
21821  2021021310        55.041518  237  233    4  16.393443  76.888889   
21822  2021021311        42.304718  202  309 -107  14.070352  74.803150   
21823  2021021311        50.011762  262  247   15  24.609375  79.003559   
21824  2021021312        49.649987  213  285  -72  14.611872  74.660633   
21825  2021021312        48.244085  214  261  -47  19.158879  85.135135   

             Sh%        Sv%         PDO  ...  Seattle Kraken  St. Louis Blues  \
0      50.000000  

<h1> Test and Training Data (Currently including Seattle, 19/20, 20/21)

In [4]:
# 2013 - 2018 = training
#print(basicGameStatsSeasonCumulativeDf)
train = basicGameStatsSeasonCumulativeDf[:14924]
#print(basicGameStatsSeasonCumulativeDf)
xTrain = train.drop(['GameID','Label'], axis=1).to_numpy()
#print(xTrain)
#print('----------------------------')
yTrain = train.iloc[:, 12].to_numpy()
yTrain = yTrain.astype('int')
#print(xTrain)
#print(yTrain)


# 2021 = test
test = basicGameStatsSeasonCumulativeDf[19202:]
xTest = test.drop(['GameID','Label'], axis=1).to_numpy()
yTest = test.iloc[:, 12].to_numpy()
yTest = yTest.astype('int')

<h1> Normalised Data

In [6]:
from sklearn.preprocessing import StandardScaler

scalar = StandardScaler()
xTrainNorm = scalar.fit(xTrain).transform(xTrain)
xTestNorm = scalar.transform(xTest)

<h1> Decision Tree

<h3> Untuned

In [7]:
from sklearn import tree
from sklearn.model_selection import cross_val_score


classifier = tree.DecisionTreeClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))
print('Depth = ' + str(classifier.get_depth()))

5-fold avg = 0.5156802556122885
Accuracy = 0.5030487804878049
Depth = 53


<h3> Tuned

In [17]:
tunedClassifier = tree.DecisionTreeClassifier()
tunedClassifier.get_params()



{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

<h1> Gaussian Naive Bayes

In [7]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_val_score

gnb = GaussianNB()

print("5-fold avg = " + str(cross_val_score(gnb, xTrain, yTrain, cv=5).mean()))

# Train classifier
gnb.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(gnb.score(xTest, yTest)))

5-fold avg = 0.5452949735271533
Accuracy = 0.53125


<h1> Support Vector Machine

<h3> Untuned

In [25]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score


# Normalise data
pipe = make_pipeline(StandardScaler(), SVC())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)


# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))



clf = SVC()

clf.fit(xTrainNorm, yTrain)
clf.predict(xTestNorm)

print('Accuracy non pipe = ' + str(clf.score(xTestNorm, yTest)))

5-fold avg = 0.5505894530741285
Accuracy = 0.551829268292683
Accuracy non pipe = 0.551829268292683


<h3> Tuned

In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

tunedClf = SVC()
tunedClf.get_params()

param_grid = {'C': [0.1, 1, 10, 100],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['linear', 'rbf']}

grid = GridSearchCV(tunedClf, param_grid, refit=True, verbose=3)

grid.fit(xTrainNorm, yTrain)
grid.predict(xTestNorm)
print('Accuracy = ' + str(grid.score(xTestNorm, yTest)))

Fitting 5 folds for each of 50 candidates, totalling 250 fits
[CV 1/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.543 total time=   9.9s
[CV 2/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.524 total time=   9.9s
[CV 3/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.544 total time=  10.1s
[CV 4/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.552 total time=  10.0s
[CV 5/5] END .....C=0.1, gamma=1, kernel=linear;, score=0.524 total time=   9.8s
[CV 1/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.503 total time=  14.6s
[CV 2/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.501 total time=  14.8s
[CV 3/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.520 total time=  14.8s
[CV 4/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.518 total time=  14.8s
[CV 5/5] END ........C=0.1, gamma=1, kernel=rbf;, score=0.509 total time=  14.8s
[CV 1/5] END ...C=0.1, gamma=0.1, kernel=linear;, score=0.543 total time=  10.0s
[CV 2/5] END ...C=0.1, gamma=0.1, kernel=linear

<h1> Neural Network

<h3> Untuned

In [9]:
from sklearn.neural_network import MLPClassifier

# Normalise data
pipe = make_pipeline(StandardScaler(), MLPClassifier())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))

/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,
/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multi

5-fold avg = 0.5129314804585932
Accuracy = 0.5038109756097561


/opt/conda/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:696: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  ConvergenceWarning,


<h3> Tuned

In [ ]:
from sklearn.neural_network import MLPClassifier

tunedClf = MLPClassifier()
tunedClf.get_params()

param_grid = {'alpha': [1, 0.1, 0.01, 0.001],
              'learning_rate': [1, 0.1, 0.01, 0.001, 0.0001]}

grid = GridSearchCV(tunedClf, param_grid, refit=True, verbose=3)

grid.fit(xTrainNorm, yTrain)
grid.predict(xTestNorm)
print('Accuracy = ' + str(grid.score(xTestNorm, yTest)))

<h1> Stochastic Gradient Descent

In [10]:
from sklearn.linear_model import SGDClassifier

# Normalise data
pipe = make_pipeline(StandardScaler(), SGDClassifier())

print("5-fold avg = " + str(cross_val_score(pipe, xTrain, yTrain, cv=5).mean()))

# Train classifier
pipe.fit(xTrain, yTrain)

# Predict class of test examples
print('Accuracy = ' + str(pipe.score(xTest, yTest)))

5-fold avg = 0.5291480189149501
Accuracy = 0.5430640243902439


<h1> AdaBoost

In [11]:
from sklearn.ensemble import AdaBoostClassifier

classifier = AdaBoostClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))


5-fold avg = 0.5473059219241875
Accuracy = 0.571265243902439


<h1> Gradient Boosting

In [12]:
from sklearn.ensemble import GradientBoostingClassifier

classifier = GradientBoostingClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))

5-fold avg = 0.5436206277140843
Accuracy = 0.5682164634146342


<h1> XGBoost 

In [13]:
import xgboost as xgb

classifier = xgb.XGBClassifier()

print("5-fold avg = " + str(cross_val_score(classifier, xTrain, yTrain, cv=5).mean()))

# Train the classifier
classifier.fit(xTrain, yTrain)

# Predict class of test examples
classifier.predict(xTest)
print('Accuracy = ' + str(classifier.score(xTest, yTest)))

5-fold avg = 0.523653365127694
Accuracy = 0.5327743902439024
